In [1]:
import scrapy
import pandas as pd
import numpy as np
import scipy as sp
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import logging

In [2]:
# Now we define the subclass of scrapy.Spider

class CarSalesSpider(scrapy.Spider):
    name = "carsales_spider" # required
    start_urls = ['http://carsalesbase.com/us-car-sales-data/'] # also required
    custom_settings = {
        'FEED_FORMAT':'csv',
        'FEED_URI':'carsales.csv',
        'DOWNLOAD_DELAY':1
    }
    
    # The first parser 
    def parse(self, response):
        BRAND_SELECTOR = '//a[starts-with(@href, "http://carsalesbase.com/us-car-sales-data/")]'
        for CarBrand in response.xpath(BRAND_SELECTOR):  # cycle over all Make links
            BRANDPATH = CarBrand.xpath('./@href').extract_first()
            yield response.follow(BRANDPATH, self.parse_brand)

    def parse_brand(self, response):
        MODEL_SELECTOR = '//a[starts-with(@href, "http://carsalesbase.com/us-car-sales-data/")]'
        for CarModel in response.xpath(MODEL_SELECTOR):  # cycle over all Model links
            MODELPATH = CarModel.xpath('./@href').extract_first()
            yield response.follow(MODELPATH, self.parse_annual_sales)
            #yield{ 'Link': CarModel.xpath('./@href').extract_first(),}

    def parse_annual_sales(self, response):
        YEAR_SELECTOR = '//*[@id="repdiv2"]' # unfortunately this selector grabs both tables since they are repeated
        Yearselect=response.xpath(YEAR_SELECTOR)
        #Make = Yearselect.xpath('./table/tbody/tr[1]/td[2]/h5[1]/text()').extract_first()
        #Model = Yearselect.xpath('./table/tbody/tr[1]/td[2]/h5[2]/text()').extract_first()
        i=0
        for firstentry in Yearselect.xpath('.//h5'):
            if i==0:
                Make = firstentry.xpath('./text()').extract_first()
                i+=1
            else:
                Model = firstentry.xpath('./text()').extract_first()
        #MakeModel = response.xpath('//title/text()').extract_first()
        #Make, Model = MakeModel.split(' ', 2)[0:2] #split the string that contains make and model as first two words
        # now look at every row for car sales data
        for entry in Yearselect.xpath('.//tr'):
            yield{
                'Make':Make,
                'Model':Model,
                'Year':entry.xpath('./td[1]/text()').extract_first(),
                'NumSold':entry.xpath('./td[2]/text()').extract_first(),
            }


In [3]:
process = CrawlerProcess(get_project_settings())

process.crawl(CarSalesSpider)
process.start()

2018-08-01 11:59:48 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-08-01 11:59:48 [scrapy.utils.log] INFO: Versions: lxml 4.2.3.0, libxml2 2.9.5, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 18.7.0, Python 2.7.14 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:34:40) [MSC v.1500 64 bit (AMD64)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2o  27 Mar 2018), cryptography 2.0.3, Platform Windows-10-10.0.17134
2018-08-01 11:59:48 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'carsales.csv', 'DOWNLOAD_DELAY': 1}
2018-08-01 11:59:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-08-01 11:59:48 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTim

2018-08-01 12:00:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-integra/>
{'NumSold': u'64.755', 'Make': u'Acura', 'Model': u'Integra', 'Year': u'1991'}
2018-08-01 12:00:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-integra/>
{'NumSold': u'83.599', 'Make': u'Acura', 'Model': u'Integra', 'Year': u'1990'}
2018-08-01 12:00:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-integra/>
{'NumSold': u'77.423', 'Make': u'Acura', 'Model': u'Integra', 'Year': u'1989'}
2018-08-01 12:00:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-integra/>
{'NumSold': u'57.468', 'Make': u'Acura', 'Model': u'Integra', 'Year': u'1988'}
2018-08-01 12:00:11 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-integra/>
{'NumSold': u'54.757', 'Make'

2018-08-01 12:00:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/hummer/hummer-h3/>
{'NumSold': u'33.140', 'Make': u'Hummer ', 'Model': u'H3', 'Year': u'2005'}
2018-08-01 12:00:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://carsalesbase.com/us-car-sales-data/honda/honda-ev-plus/> (referer: http://carsalesbase.com/us-car-sales-data/honda/)
2018-08-01 12:00:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/honda/honda-ev-plus/>
{'NumSold': u'\n', 'Make': u'Honda', 'Model': u'EV Plus', 'Year': None}
2018-08-01 12:00:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/honda/honda-ev-plus/>
{'NumSold': u'2', 'Make': u'Honda', 'Model': u'EV Plus', 'Year': u'2000'}
2018-08-01 12:00:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/honda/honda-ev-plus/>
{'NumSold': u'62', 'Make': u'Honda', 'Model': u'EV Plus', 'Year': 

2018-08-01 12:00:17 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/hyundai/hyundai-genesis/>
{'NumSold': u'29.122', 'Make': u'Hyundai', 'Model': u'Genesis', 'Year': u'2010'}
2018-08-01 12:00:17 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/hyundai/hyundai-genesis/>
{'NumSold': u'21.889', 'Make': u'Hyundai', 'Model': u'Genesis', 'Year': u'2009'}
2018-08-01 12:00:17 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/hyundai/hyundai-genesis/>
{'NumSold': u'6.167', 'Make': u'Hyundai', 'Model': u'Genesis', 'Year': u'2008'}
2018-08-01 12:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://carsalesbase.com/us-car-sales-data/gmc/gmc-caballero/> (referer: http://carsalesbase.com/us-car-sales-data/gmc/)
2018-08-01 12:00:18 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/gmc/gmc-caballero/>
{'NumSold': u'\n', 'Make': u'GMC', 'M

2018-08-01 12:00:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/genesis/genesis-g90/>
{'NumSold': u'\n', 'Make': u'Genesis', 'Model': u'G90', 'Year': None}
2018-08-01 12:00:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/genesis/genesis-g90/>
{'NumSold': u'4.398', 'Make': u'Genesis', 'Model': u'G90', 'Year': u'2017'}
2018-08-01 12:00:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/genesis/genesis-g90/>
{'NumSold': u'782', 'Make': u'Genesis', 'Model': u'G90', 'Year': u'2016'}
2018-08-01 12:00:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/genesis/genesis-g90/>
{'NumSold': u'\n', 'Make': u'Genesis', 'Model': u'G90', 'Year': None}
2018-08-01 12:00:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/genesis/genesis-g90/>
{'NumSold': u'4.398', 'Make': u'Genesis', 'Model': u'G9

...

2018-08-01 12:22:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-tl/>
{'NumSold': u'67.033', 'Make': u'Acura', 'Model': u'TL', 'Year': u'2000'}
2018-08-01 12:22:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-tl/>
{'NumSold': u'56.566', 'Make': u'Acura', 'Model': u'TL', 'Year': u'1999'}
2018-08-01 12:22:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-tl/>
{'NumSold': u'31.883', 'Make': u'Acura', 'Model': u'TL', 'Year': u'1998'}
2018-08-01 12:22:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-tl/>
{'NumSold': u'23.151', 'Make': u'Acura', 'Model': u'TL', 'Year': u'1997'}
2018-08-01 12:22:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-tl/>
{'NumSold': u'24.700', 'Make': u'Acura', 'Model': u'TL', 'Year': u'1996'}


2018-08-01 12:22:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-cl/>
{'NumSold': u'283', 'Make': u'Acura', 'Model': u'CL', 'Year': u'2004'}
2018-08-01 12:22:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-cl/>
{'NumSold': u'6.593', 'Make': u'Acura', 'Model': u'CL', 'Year': u'2003'}
2018-08-01 12:22:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-cl/>
{'NumSold': u'12.072', 'Make': u'Acura', 'Model': u'CL', 'Year': u'2002'}
2018-08-01 12:22:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-cl/>
{'NumSold': u'18.993', 'Make': u'Acura', 'Model': u'CL', 'Year': u'2001'}
2018-08-01 12:22:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/acura/acura-cl/>
{'NumSold': u'24.677', 'Make': u'Acura', 'Model': u'CL', 'Year': u'2000'}
2018

2018-08-01 12:22:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mitsubishi/mitsubishi-eclipse/>
{'NumSold': u'19.361', 'Make': u'Mitsubishi', 'Model': u'Eclipse', 'Year': u'2004'}
2018-08-01 12:22:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mitsubishi/mitsubishi-eclipse/>
{'NumSold': u'38.875', 'Make': u'Mitsubishi', 'Model': u'Eclipse', 'Year': u'2003'}
2018-08-01 12:22:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mitsubishi/mitsubishi-eclipse/>
{'NumSold': u'72.041', 'Make': u'Mitsubishi', 'Model': u'Eclipse', 'Year': u'2002'}
2018-08-01 12:22:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mitsubishi/mitsubishi-eclipse/>
{'NumSold': u'70.351', 'Make': u'Mitsubishi', 'Model': u'Eclipse', 'Year': u'2001'}
2018-08-01 12:22:21 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sa

2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'6.305', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2012'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'5.674', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2011'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'6.370', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2010'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'7.917', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2009'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumS

2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'9.356', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2004'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'10.920', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2003'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'14.392', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2002'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'NumSold': u'16.486', 'Make': u'Mazda', 'Model': u'MX-5 Miata', 'Year': u'2001'}
2018-08-01 12:22:23 [scrapy.core.scraper] DEBUG: Scraped from <200 http://carsalesbase.com/us-car-sales-data/mazda/mazda-mx-5-miata/>
{'N